Creamos las funciones de Endpoints
Importamos las librerías 

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings('ignore')
from typing import Dict, List



Las funciones las voy a realizar con el archivo parquet

Cargo el Dataset de juegos 

In [2]:
df_games = pd.read_parquet("Data/games_reviews.parquet") # Cargamos el dataset de juegos
df_games.head()

,app_name,release_date,price,item_id,developer,release_year,genres_clean,user_id,posted,recommend,posted_year,sentiment_analysis
0,"Pirates, Vikings, and Knights II",2010-02-12 00:00:00.000000000,0.0,17570,Octoshark Studios,2010,Action,76561198010653835,2015-04-07,True,2015,2
1,"Pirates, Vikings, and Knights II",2010-02-12 00:00:00.000000000,0.0,17570,Octoshark Studios,2010,Action,76561198075581042,2014-03-09,True,2014,2
2,"Pirates, Vikings, and Knights II",2010-02-12 00:00:00.000000000,0.0,17570,Octoshark Studios,2010,Action,GameBrak,2015-06-26,True,2015,2
3,"Pirates, Vikings, and Knights II",2010-02-12 00:00:00.000000000,0.0,17570,Octoshark Studios,2010,Action,CjDoesIntros,2014-02-05,True,2014,1
4,"Pirates, Vikings, and Knights II",2010-02-12 00:00:00.000000000,0.0,17570,Octoshark Studios,2010,Action,76561198075920681,2015-06-07,True,2015,1


In [3]:
df_user_items = pd.read_parquet("Data/users_items.parquet") # Cargamos el dataset de items
df_user_items.head()

,item_id,item_name,steam_id,items_count,user_id,playtime_hours
0,10,Counter-Strike,76561197970982479,277,76561197970982479,0.10
1,30,Day of Defeat,76561197970982479,277,76561197970982479,0.12
2,300,Day of Defeat: Source,76561197970982479,277,76561197970982479,78.88
3,240,Counter-Strike: Source,76561197970982479,277,76561197970982479,30.88
4,3830,Psychonauts,76561197970982479,277,76561197970982479,5.55


In [5]:
df_games_and_reviews = pd.read_parquet("Data/games_reviews.parquet") # Cargamos el dataset de reviews
df_games_and_reviews.head()

,app_name,release_date,price,item_id,developer,release_year,genres_clean,user_id,posted,recommend,posted_year,sentiment_analysis
0,"Pirates, Vikings, and Knights II",2010-02-12 00:00:00.000000000,0.0,17570,Octoshark Studios,2010,Action,76561198010653835,2015-04-07,True,2015,2
1,"Pirates, Vikings, and Knights II",2010-02-12 00:00:00.000000000,0.0,17570,Octoshark Studios,2010,Action,76561198075581042,2014-03-09,True,2014,2
2,"Pirates, Vikings, and Knights II",2010-02-12 00:00:00.000000000,0.0,17570,Octoshark Studios,2010,Action,GameBrak,2015-06-26,True,2015,2
3,"Pirates, Vikings, and Knights II",2010-02-12 00:00:00.000000000,0.0,17570,Octoshark Studios,2010,Action,CjDoesIntros,2014-02-05,True,2014,1
4,"Pirates, Vikings, and Knights II",2010-02-12 00:00:00.000000000,0.0,17570,Octoshark Studios,2010,Action,76561198075920681,2015-06-07,True,2015,1


Función PlayTimeGamer

La función PlayTimeGamer sirve para calcular y encontrar el tiempo total de juego registrado en horas para juegos específicos en un conjunto de datos. Esta función ayuda a identificar cuántas horas han sido jugadas en total para cada juego, lo que puede ser útil para analizar la popularidad o el tiempo dedicado a juegos individuales en una plataforma de juegos como Steam.

In [12]:
def PlayTimeGenre(genero, df_games: pd.DataFrame, df_user_items: pd.DataFrame, half_size: int):
    # Convertimos el género proporcionado a minúsculas para una comparación sin distinción de mayúsculas
    genero = genero.lower()

    # Filtramos los juegos por género de manera más flexible
    df_filtered = df_games[df_games['genres_clean'].str.lower().str.contains(fr'\b{genero}\b', na=False)]

    if not df_filtered.empty:
        # Convertimos 'item_id' a tipo de dato object en ambos DataFrames
        df_user_items['item_id'] = df_user_items['item_id'].astype(str)
        df_filtered['item_id'] = df_filtered['item_id'].astype(str)

        # Filtramos el DataFrame `df_games` para obtener solo la mitad
        df_filtered = df_filtered.sample(frac=0.5, random_state=42)

        # Fusionamos los DataFrames `df_user_items` y `df_filtered` por la columna `item_id`
        df_merged = pd.merge(df_user_items, df_filtered[['item_id', 'release_year']], left_on='item_id', right_on='item_id')

        # Verificamos si la longitud de df_filtered es mayor a cero, es decir, se encontraron géneros
        result = {"Año con más horas jugadas para el género {}: {}".format(genero.capitalize(), df_merged.groupby('release_year')['playtime_hours'].sum().idxmax())}
    else:
        result = {"Año con más horas jugadas para género {}: {}".format(genero.capitalize(), "Género no encontrado en la base de datos")}

    return result

In [13]:
result = PlayTimeGenre("Sports", df_games, df_user_items, half_size=500)
print(result)

{'Año con más horas jugadas para el género Sports: 2015'}


Función UsersWorstDeveloper

Esta función tiene como propósito identificar y proporcionar información sobre los desarrolladores de juegos menos recomendados por los usuarios en un año específico. Su objetivo es encontrar las empresas de desarrollo cuyos juegos hayan recibido críticas negativas por parte de los usuarios durante un periodo de tiempo determinado, lo que puede ser útil para evaluar la calidad o la recepción de los productos de estos estudios en un año en particular.



In [14]:
def UsersWorstDeveloper(año: int, df_games_and_reviews: pd.DataFrame):
    # Verificamos si el año está dentro del rango esperado
    rango_aceptado = range(2010, 2016)
    if año not in rango_aceptado:
        return {"message": "Mi base de datos solo tiene registros entre 2010 y 2015"}

    # Filtramos por comentarios no recomendados y sentiment_analysis negativo
    df_filtered = df_games_and_reviews[(df_games_and_reviews['recommend'] == False) & (df_games_and_reviews['sentiment_analysis'] == 0)]

    # Filtramos por el año deseado
    df_filtered_year = df_filtered[df_filtered['release_year'] == año]

    # Si no hay datos para el año, retornamos mensaje
    if not df_filtered_year.empty:
        # Obtener los top 3 desarrolladores con menos recomendaciones
        top_developers = df_filtered_year['developer'].value_counts().head(3).reset_index()
        top_developers = top_developers.rename(columns={'index': 'Puesto 1', 'developer': 'Desarrollador'})

        # Modificamos la estructura del resultado
        result = [{"Puesto {}".format(i + 1): desarrollador} for i, desarrollador in enumerate(top_developers['Desarrollador'])]
    else:
        result = {"No hay juegos no recomendados para el año {}".format(año)}

    return result

In [15]:
UsersWorstDeveloper(2010, df_games_and_reviews)

[{'Puesto 1': 4}, {'Puesto 2': 3}, {'Puesto 3': 3}]

Función UsersRecommend 

La función UsersRecommend está diseñada para encontrar y devolver los juegos más recomendados dentro de un rango de años específico en una base de datos de juegos. Ayuda a identificar los juegos que han recibido más recomendaciones positivas o neutrales, ayudando a los usuarios a descubrir los títulos más populares o bien valorados en ese período de tiempo.

In [16]:
def UsersRecommend(año: int, df_games_and_reviews: pd.DataFrame):
    # Verificamos si el año está dentro del rango esperado
    rango_aceptado = range(2007, 2010)
    if año not in rango_aceptado:
        return {"message": "Mi base de datos tiene registros entre el 2007 y 2015"}

    # Filtramos por el año deseado
    df_filtered = df_games_and_reviews[df_games_and_reviews['release_year'] == año]

    # Filtramos por comentarios recomendados y sentiment_analysis positivo/neutral
    df_filtered = df_filtered[(df_filtered['recommend'] == True) & df_filtered['sentiment_analysis'].isin(['Positive', 'Neutral'])]

    # Obtenemos el top 3 de juegos recomendados
    top_games = df_filtered['app_name'].value_counts().head(3).reset_index()
    top_games = top_games.rename(columns={'index': 'Juego', 'app_name': 'Recomendaciones'})

   
  # Modificamos la estructura del resultado
    resultado = [{"Puesto {}".format(i + 1): juego, 'Recomendaciones': count} for i, (juego, count) in enumerate(zip(top_games['Juego'], top_games['Recomendaciones']))]

    return resultado  # Devolvemos la variable 'result' que contiene el resultado de la función 
     


In [17]:
# Llamamos a la función UsersRecommend con un año específico y el DataFrame correspondiente
resultado = UsersRecommend(2010, df_games_and_reviews)

# Imprimimos el resultado
print(resultado)

{'message': 'Mi base de datos tiene registros entre el 2007 y 2015'}


Función UserWorsDeveloper

La función UserWorsDeveloper tiene como objetivo encontrar y proporcionar información sobre las desarrolladoras de juegos menos recomendadas por los usuarios en un año específico. 

In [18]:
def UsersWorstDeveloper(año: int, df_games_and_reviews: pd.DataFrame):
    # Verificamos si el año está dentro del rango esperado
    rango_aceptado = range(2010, 2016)
    if año not in rango_aceptado:
        return {"message": "Mi base de datos solo tiene registros entre 2010 y 2015"}

    # Filtramos por comentarios no recomendados y sentiment_analysis negativo
    df_filtered = df_games_and_reviews[(df_games_and_reviews['recommend'] == False) & (df_games_and_reviews['sentiment_analysis'] == 0)]

    # Filtramos por el año deseado
    df_filtered_year = df_filtered[df_filtered['release_year'] == año]

    # Si no hay datos para el año, retornamos mensaje
    if not df_filtered_year.empty:
        # Obtener los top 3 desarrolladores con menos recomendaciones
        top_developers = df_filtered_year['developer'].value_counts().head(3).reset_index()
        top_developers = top_developers.rename(columns={'index': 'Puesto 1', 'developer': 'Desarrollador'})

        # Modificamos la estructura del resultado
        result = [{"Puesto {}".format(i + 1): desarrollador} for i, desarrollador in enumerate(top_developers['Desarrollador'])]
    else:
        result = {"No hay juegos no recomendados para el año {}".format(año)}

    return result

In [19]:
UsersWorstDeveloper(2015, df_games_and_reviews)

[{'Puesto 1': 15}, {'Puesto 2': 10}, {'Puesto 3': 7}]

Funcion sentiment_analysis

In [20]:
def sentiment_analysis(empresa_desarrolladora: str, df_games_and_reviews: pd.DataFrame):
    # Filtramos por desarrolladora
    df_filtered_developer = df_games_and_reviews[df_games_and_reviews['developer'] == empresa_desarrolladora]

    # Verificamos que haya datos para la desarrolladora
    if not df_filtered_developer.empty:
        # Contamos los sentimientos y mapeamos el número del sentimiento a su etiqueta correspondiente
        sentiment_counts = df_filtered_developer['sentiment_analysis'].value_counts().reset_index(drop=True)
        sentiment_mapping = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
        sentiment_counts.index = sentiment_counts.index.map(sentiment_mapping)
        
        result = {empresa_desarrolladora: sentiment_counts.to_dict()}
    else:
        result = 'No cuento con los registros de esa empresa en mi base de datos'

    return result

In [21]:
sentiment_analysis('Treyarch', df_games_and_reviews)

{'Treyarch': {'Negative': 155, 'Neutral': 121, 'Positive': 37}}